<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/C3TR_Adapterv2_Japanese_English_Translation_sample_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日英・英日翻訳サンプルコード(Japanese-English English-Japanese Translation sample code)


C3TR-Adapter Version2を使って日本語を英語、英語を翻訳するサンプルコードです。  
Sample code to translate Japanese to English and English to Japanese using C3TR-Adapter Version2.

上段メニューの「ランタイム」→「すべてのセルを実行」で実行してください  
Please execute it by clicking "Runtime" -> "Execute All Cells" in the upper menu  


もしくは各セルの[ ]の部分をクリックして実行してください  
Or click on the [ ] part of each cell to execute it.  


In [1]:
%%bash
pip install peft==0.10.0 bitsandbytes==0.43.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 23.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-man

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

model_id = "unsloth/gemma-7b-bnb-4bit"
peft_model_id = "webbigdata/C3TR-Adapter"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
model = PeftModel.from_pretrained(model = model, model_id = peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


...

adapter_model.safetensors:   0%|          | 0.00/800M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [8]:
def trans(my_str):
    input_ids = tokenizer(my_str, return_tensors="pt",
        padding=True, max_length=1600, truncation=True).input_ids.cuda()

    # Translation
    generated_ids = model.generate(input_ids=input_ids,
        num_beams=3, max_new_tokens=800, do_sample=True, temperature=0.5, top_p=0.3,
        use_cache=True,
        )
    full_outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return full_outputs[0].split("### Response:\n")[-1].strip()

In [12]:
ret = trans("""You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

### Instruction:
Translate Japanese to English.
When translating, please use the following hints:
[writing_style: casual]
[波平部長: Manager Namihei]

### Input:
ご注文よろしいでしょうか？
私は日替わり定食で
波平部長は？
部長はウナギです"

### Response:
""")
print(ret)

ret = trans("""You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

### Instruction:

Translate English to Japanese.
When translating, please use the following hints.
[writing_style: casual, historical drama style]
[Yabushige: 藪重]
[sentence_first_person_and_ending: 拙者, ござる]

### Input:
I am Yabushige, leader of izu samurai.

### Response:
""")
print(ret)

Is that your order?
I'll have the daily special.
What about you, Manager Namihei?
The manager will have eel.
拙者は藪重、伊豆の侍の首領でござる。


## 留意事項 Attention
**Do not save this adapter merged with the base model**, as there exists a bug that reduces performance when saving this adapter merged with the model.    

このアダプターをモデルとマージして保存すると性能が下がってしまう不具合が存在するため、**ベースモデルとマージして保存しないでください**

## 謝辞 Acknowledgment

### Referenced models.
Original Base Model  
google/gemma-7b  
https://huggingface.co/google/gemma-7b

Base Model  
unsloth/gemma-7b-bnb-4bit  
https://huggingface.co/unsloth/gemma-7b-bnb-4bit

QLoRA Adapter  
webbigdata/C3TR-Adapter  
https://huggingface.co/webbigdata/C3TR-Adapter


This adapter was trained with Unsloth.  
https://github.com/unslothai/unsloth  

その他、[ALMA](https://arxiv.org/abs/2309.11674)をはじめ、コミュニティの皆さんからヒントを貰っています。ありがとう  
Other tips I have received from [ALMA](https://arxiv.org/abs/2309.11674) and others in the community. Thank you.  

このスクリプトは[webbigdata](https://webbigdata.jp/)によって作成されました  
This script was created by [webbigdata](https://webbigdata.jp/).  